# VIX

## Set Up

In [1]:
dataset_file_name = "^VIX.csv"
path_data_original = "../Data/original/"
date_original_format = "yyyy-mm-dd"
original_value_column = 4 # High
last_position_change = "2020-09-30"

#include("../Julia/MarketIndicators.jl") 
include("../Julia/functions.jl")

println("Run this file from the directory where this file is located.")
pwd()


Run this file from the directory where this file is located.


"/Users/uki/REPOS/MarketIndicators.jl/src/notebooks_indicators"

## Read Data

In [2]:
#data = available_datasets() # uncomment to see all available datasets

df = fetch_dataset(dataset_file_name, date_original_format , path_data_original )
record_count = size(df)[1]
println("data dimentions: ", size(df) )

#using Statistics
#describe(df)

../Data/original/^VIX.csv
data dimentions: (7790, 7)


### Sample the data

In [3]:
columns = preview_data(df) # uncomment to see the data
println()

7×7 DataFrame
│ Row │ Date       │ Open    │ High    │ Low     │ Close   │ Adj Close │
│     │ Date       │ Float64 │ Float64 │ Float64 │ Float64 │ Float64   │
├─────┼────────────┼─────────┼─────────┼─────────┼─────────┼───────────┤
│ 1   │ 1990-01-02 │ 17.24   │ 17.24   │ 17.24   │ 17.24   │ 17.24     │
│ 2   │ 1995-02-17 │ 11.44   │ 12.06   │ 11.29   │ 11.71   │ 11.71     │
│ 3   │ 2000-04-10 │ 25.44   │ 26.04   │ 24.97   │ 25.99   │ 25.99     │
│ 4   │ 2005-06-10 │ 12.16   │ 12.4    │ 11.9    │ 11.96   │ 11.96     │
│ 5   │ 2010-08-06 │ 23.34   │ 23.89   │ 21.72   │ 21.74   │ 21.74     │
│ 6   │ 2015-10-02 │ 23.99   │ 24.47   │ 20.35   │ 20.94   │ 20.94     │
│ 7   │ 2020-11-27 │ 21.52   │ 21.6    │ 19.51   │ 20.84   │ 20.84     │

│ Row │ Volume │
│     │ Int64  │
├─────┼────────┤
│ 1   │ 0      │
│ 2   │ 0      │
│ 3   │ 0      │
│ 4   │ 0      │
│ 5   │ 0      │
│ 6   │ 0      │
│ 7   │ 0      │
 Dataset dimentions: (7790, 7)

 variable columns:
1 Date
2 Open
3 High
4 Low
5 Close

## Rata Die

In [4]:
col_ind = 1
insertcols!(df, col_ind, :Rata_Die => zeros(Int64, record_count); makeunique = true )

update_rata_die!(df, 1, 2)
#first(df, 6)

columns = preview_data(df, 3)
println()

3×8 DataFrame
│ Row │ Rata_Die │ Date       │ Open    │ High    │ Low     │ Close   │
│     │ Int64    │ Date       │ Float64 │ Float64 │ Float64 │ Float64 │
├─────┼──────────┼────────────┼─────────┼─────────┼─────────┼─────────┤
│ 1   │ 726469   │ 1990-01-02 │ 17.24   │ 17.24   │ 17.24   │ 17.24   │
│ 2   │ 730221   │ 2000-04-11 │ 27.67   │ 28.08   │ 26.64   │ 27.25   │
│ 3   │ 733994   │ 2010-08-10 │ 23.3    │ 24.24   │ 22.17   │ 22.37   │

│ Row │ Adj Close │ Volume │
│     │ Float64   │ Int64  │
├─────┼───────────┼────────┤
│ 1   │ 17.24     │ 0      │
│ 2   │ 27.25     │ 0      │
│ 3   │ 22.37     │ 0      │
 Dataset dimentions: (7790, 8)

 variable columns:
1 Rata_Die
2 Date
3 Open
4 High
5 Low
6 Close
7 Adj Close
8 Volume



### Draw Plots

### set the original value column
- go back to the top "Setup" section and choose which column should be the "Orignal value"
- re-run the notebook

## Extract right columns

In [5]:
# Extract only the columns to be kept

using DataFrames
df = DataFrame( Day      = df[:,1],                     # 1 
                Date     = df[:,2],                     # 2 
                Value    = df[:,original_value_column], # 3 
                Original = df[:,original_value_column]  # 4 
               )

columns = preview_data(df)
println()

7×4 DataFrame
│ Row │ Day    │ Date       │ Value   │ Original │
│     │ Int64  │ Date       │ Float64 │ Float64  │
├─────┼────────┼────────────┼─────────┼──────────┤
│ 1   │ 726469 │ 1990-01-02 │ 17.24   │ 17.24    │
│ 2   │ 728341 │ 1995-02-17 │ 12.06   │ 12.06    │
│ 3   │ 730220 │ 2000-04-10 │ 26.04   │ 26.04    │
│ 4   │ 732107 │ 2005-06-10 │ 12.4    │ 12.4     │
│ 5   │ 733990 │ 2010-08-06 │ 23.89   │ 23.89    │
│ 6   │ 735873 │ 2015-10-02 │ 24.47   │ 24.47    │
│ 7   │ 737756 │ 2020-11-27 │ 21.6    │ 21.6     │
 Dataset dimentions: (7790, 4)

 variable columns:
1 Day
2 Date
3 Value
4 Original



## Quantize the values

- Quantization is a process of noramalizing the data
- I have decided to normalize the data for Int8 as I might try use Google Coral NPU
- minimum = -128.0
- maximum = 127.0

In [6]:
# Quantize the values

data_original = df[:, 4] # keep original for display comparison later

quantize_column!(df, 3)

columns = preview_data(df)
println()

bias for 0 value: -0.11605584786905819
Normalization was performend using formula y=(((x-min)/(max-min))*255)-128 
minimum = 9.31
maximum = 89.529999
Save these values for later to run model preditions
7×4 DataFrame
│ Row │ Day    │ Date       │ Value   │ Original │
│     │ Int64  │ Date       │ Float64 │ Float64  │
├─────┼────────┼────────────┼─────────┼──────────┤
│ 1   │ 726469 │ 1990-01-02 │ -103.0  │ 17.24    │
│ 2   │ 728341 │ 1995-02-17 │ -119.0  │ 12.06    │
│ 3   │ 730220 │ 2000-04-10 │ -75.0   │ 26.04    │
│ 4   │ 732107 │ 2005-06-10 │ -118.0  │ 12.4     │
│ 5   │ 733990 │ 2010-08-06 │ -82.0   │ 23.89    │
│ 6   │ 735873 │ 2015-10-02 │ -80.0   │ 24.47    │
│ 7   │ 737756 │ 2020-11-27 │ -89.0   │ 21.6     │
 Dataset dimentions: (7790, 4)

 variable columns:
1 Day
2 Date
3 Value
4 Original



## Fill in Empty Dates

In [7]:
populate_missing_dates!(df)

,Day,Date,Value,Original
,Int64,Date,Float64,Float64
1,726469,1990-01-02,-103.0,17.24
2,726470,1990-01-03,-100.0,18.19
3,726471,1990-01-04,-96.0,19.22
4,726472,1990-01-05,-94.0,20.11
5,726475,1990-01-08,-93.0,20.26
6,726476,1990-01-09,-87.0,22.2
7,726477,1990-01-10,-86.0,22.44
8,726478,1990-01-11,-94.0,20.05
9,726479,1990-01-12,-79.0,24.64


### Fill: Sort

In [8]:
df = sort(df, [:Day]);
count = size(df)[1]
#first(df, 6)
# columns = preview_data(df)
println()

### Fill: Draw Plots
- if the indicator is updated only preiodically (bi-weekly, monthly, quarterly) the the graph will appear blocky

## Averages

In [9]:
column_to_average = 3
averages005 = calculate_average(df, 5,   column_to_average )
averages030 = calculate_average(df, 30,  column_to_average )
averages090 = calculate_average(df, 90,  column_to_average )
averages180 = calculate_average(df, 180, column_to_average )
averages365 = calculate_average(df, 365, column_to_average )
println()

### Insert averages to DataFrame

In [10]:
insertcols!(df, 5,  :Avg005   => averages005  , makeunique=true)
insertcols!(df, 6,  :Avg030   => averages030  , makeunique=true)
insertcols!(df, 7,  :Avg090   => averages090  , makeunique=true)
insertcols!(df, 8,  :Avg180   => averages180  , makeunique=true)
insertcols!(df, 9,  :Avg365   => averages365  , makeunique=true)

#using Statistics
#describe(df)

,Day,Date,Value,Original,Avg005,Avg030,Avg090,Avg180,Avg365
,Int64,Date,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,726469,1990-01-02,-103.0,17.24,0.0,0.0,0.0,0.0,0.0
2,726470,1990-01-03,-100.0,18.19,0.0,0.0,0.0,0.0,0.0
3,726471,1990-01-04,-96.0,19.22,0.0,0.0,0.0,0.0,0.0
4,726472,1990-01-05,-94.0,20.11,0.0,0.0,0.0,0.0,0.0
5,726473,1990-01-06,-94.0,20.11,-97.4,0.0,0.0,0.0,0.0
6,726474,1990-01-07,-94.0,20.11,-95.6,0.0,0.0,0.0,0.0
7,726475,1990-01-08,-93.0,20.26,-94.2,0.0,0.0,0.0,0.0
8,726476,1990-01-09,-87.0,22.2,-92.4,0.0,0.0,0.0,0.0
9,726477,1990-01-10,-86.0,22.44,-90.8,0.0,0.0,0.0,0.0


[back to top](#Table-of-Contents)
<hr/>

## Save DataFrame to CSV file

### Save: Describe before saving

In [13]:
using Statistics
describe(df)

,variable,mean,min,median,max,nunique,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Union…,Nothing,DataType
1,Day,732114.0,726469,732114.0,737759,,,Int64
2,Date,,1990-01-02,,2020-11-30,11291,,Date
3,Value,-93.2221,-128.0,-100.0,127.0,,,Float64
4,Original,20.2507,9.31,17.98,89.53,,,Float64
5,Avg005,-93.189,-127.2,-100.4,115.6,,,Float64
6,Avg030,-93.0046,-125.13,-100.57,69.8,,,Float64
7,Avg090,-92.6169,-123.27,-100.26,43.81,,,Float64
8,Avg180,-91.984,-122.07,-99.35,17.55,,,Float64
9,Avg365,-90.867,-120.61,-99.11,0.0,,,Float64


### Write as CSV file

In [12]:
save_dataset(df, dataset_file_name, "../Data/processed/" );